In [1]:
import sys
import os

CARTOLA_HOME = os.path.abspath('..')

if not os.path.isdir(CARTOLA_HOME):
    raise Exception('Cartola project is missing!')
    
sys.path.append(CARTOLA_HOME)

In [2]:
from cartola_game.data import load_season

season_2015 = load_season('2015')

print(season_2015)

2015 (rounds 38, players 1041)


In [3]:
from cartola_game.model import Team, Formation, Player, Position, team_is_valid

In [4]:
valid_team = [Player('a', Position.GOALKEEPER)]
valid_team += [Player('d {}'.format(i+1), Position.DEFENDER) for i in range(3)]
valid_team += [Player('m {}'.format(i+1), Position.MIDFIELDER) for i in range(4)]
valid_team += [Player('f {}'.format(i+1), Position.FORWARD) for i in range(3)]
valid_team += [Player('e', Position.COACH)]

team_is_valid(Formation.F343, valid_team)

True

In [5]:
from collections import namedtuple

PlayerState = namedtuple('PlayerState', ['player', 'status', 'price', 'price_var', 'score'])

class RoundEngine:
    
    def __init__(self, round_number, players_data):
        self.round_number = round_number
        self.players_data = players_data

    def price(self, players):
        return sum(self.players_data[p].price for p in players)

class ScoreEngine:
    
    def __init__(self, players_score):
        self.players_score = players_score

    def score(self, players):
        return sum(self.players_score[p] for p in players)

class GameEngine:
    
    def __init__(self, season_data):
        rounds = []
        scores = []
        
        for i in range(season_data.num_rounds):
            round_players = {}
            score_players = {}
            for player_data in season_data.players:
                p = player_data.player
                round_players[p] = self._round_player(player_data, i)
                score_players[p] = self._score_player(player_data, i)
            rounds.append(RoundEngine(i + 1, round_players))
            scores.append(ScoreEngine(score_players))
        
        self.rounds = rounds
        self.scores = scores
    
    def _round_player(self, player_data, i):
        player = player_data.player
        status = player_data.status[i]
        price = player_data.prices[i]
        price_var = 0.0 if i == 0 else price - player_data.prices[i - 1]
        score = 0.0 if i == 0 else player_data.scores[i - i]
        return PlayerState(player, status, price, price_var, score)
    
    def _score_player(self, player_data, i):
        return player_data.scores[i]
    
    @property
    def num_rounds(self):
        return len(self.rounds)
    
    def round_engine(self, round_number):
        return self.rounds[round_number - 1]

    def score_engine(self, round_number):
        return self.scores[round_number - 1]

class GameState(namedtuple('GameState', ['round_number', 'team', 'team_price', 'team_score', 'money'])):
    __slots__ = ()

    def __str__(self):
        out = '[ Round {} ]\n\n'.format(self.round_number)
        out += str(self.team)
        out += '\n\nTeam Price: {:.2f}\n'.format(self.team_price)
        out += 'Unused Money: {:.2f}\n\n'.format(self.money)
        out += 'Score: {:.2f}'.format(self.team_score)
        return out

class GameInstance:
    
    def __init__(self, engine, initial_money=100.0):
        self.engine = engine
        self.initial_money = initial_money
        self.states = []

    @property
    def done(self):
        return len(self.states) == self.engine.num_rounds
    
    @property
    def round_number(self):
        return len(self.states) + 1
    
    @property
    def data(self):
        return self.engine.round_engine(self.round_number - int(self.done))
    
    @property
    def total_money(self):
        if self.round_number == 1:
            return self.initial_money
        s = self.states[-1]
        return s.money + self.data.price(s.team.players)
    
    def run(self, team):
        if self.done:
            raise Exception('Game is done!')

        running_round = self.round_number

        budget = self.total_money
        team_price = self.data.price(team.players)
        free_money = budget - team_price
        if free_money < 0.0:
            raise Exception('Invalid team: insufficient budget!')

        team_score = self.engine.score_engine(running_round).score(team.players)

        s = GameState(running_round, team, team_price, team_score, free_money)
        self.states.append(s)

        return team_score

In [6]:
x = GameEngine(season_2015)

In [7]:
import random

def filter_players(players, position, max_price):
    return [p for p in players if p.player.position == position and p.price < max_price]

def sample_players(players, formation, budget):
    selected_players = []
    for pos, n in formation.positions:
        candidates = filter_players(players, pos, budget)
        if len(candidates) < n:
            return None
        for player in random.sample(candidates, n):
            budget -= player.price
            if budget < 0.0:
                return None
            selected_players.append(player.player)
    return selected_players
    
def random_team(data, budget, max_iterations=1000000):
    players_list = list(data.players_data.values())
    for _ in range(max_iterations):
        formation = random.choice(list(Formation))
        players = sample_players(players_list, formation, budget)
        if not players:
            continue
        return Team(formation, players)
    return None

data = x.round_engine(1)
team = random_team(data, 100.0)
print(team)
print('\nPrice: {:.2f}'.format(data.price(team.players)))

4-3-3

Goleiro -> Nelson de Jesus Silva
Lateral -> Ayrton Luis Ganino
Lateral -> Erick Fernando Brandão Daltro
Zagueiro -> Felipe Augusto de Almeida Monteiro
Zagueiro -> Nathan Pelae Cardoso
Meia -> Jhon Cley Jesus Silva
Meia -> Jonatan Lucca
Meia -> Sergio Antonio Soler de Oliveira Junior
Atacante -> Bruno Rangel Domingues
Atacante -> Lucca Borges Brito
Atacante -> Ruan Carlos Gomes Costa da Silva
Técnico -> Paulo Roberto Falcão

Price: 49.00


In [8]:
i = GameInstance(x)

t_score = 0.0
while not i.done:
    print('[ Round', i.round_number, ']\n')
    data = i.data
    budget = i.total_money
    team = random_team(data, budget)
    
    print(team)
    team_value = data.price(team.players)
    print('\nTeam Price: {:.2f}'.format(team_value))
    print('Unused Money: {:.2f}'.format(budget - team_value))
    
    score = i.run(team)
    t_score += score
    
    print('\nScore: {:.2f}'.format(score))
    after_money = i.total_money
    print('Money: {:.2f}'.format(after_money))
    print('Variation: {:.2f}\n'.format(after_money - budget))

print('...')
print('\nFinal Score: {:.2f}'.format(t_score))

[ Round 1 ]

4-4-2

Goleiro -> Jordi Almeida
Lateral -> Francisco Sousa dos Santos
Lateral -> Renê Rodrigues Martins
Zagueiro -> Antônio Carlos dos Santos Aguiar
Zagueiro -> Douglas Ricardo Grolli
Meia -> Juliano Real Pacheco
Meia -> Rudnei da Rosa
Meia -> Sandro Laurindo da Silva
Meia -> Ítalo Melo Oliveira
Atacante -> Everaldo Stum
Atacante -> José Fagner Silva da Luz
Técnico -> Sergio Agostinho de Oliveira Vieira

Team Price: 44.57
Unused Money: 55.43

Score: -6.80
Money: 99.18
Variation: -0.82

[ Round 2 ]

5-3-2

Goleiro -> Vitor Prada Macaneiro
Lateral -> Daniel Borges Fortunato
Lateral -> João Lucas Lima Silva
Zagueiro -> Henrique de Souza Trevisan
Zagueiro -> Rafael Marques Pinto
Zagueiro -> Rodrigo Modesto da Silva Moledo
Meia -> Icaro Cosmo da Rocha
Meia -> Wagner Ricardo Silva da Silva
Meia -> Yuri Naves Roberto
Atacante -> Jonathan Renato Barbosa
Atacante -> William Júnior Salles de Lima Souza
Técnico -> Argélico Fucks

Team Price: 22.97
Unused Money: 76.21

Score: 6.62
Mon